# Chatbot with message summarization & external DB memory

## Review

We've covered how to customize graph state schema and reducer.

We've also shown a number of tricks for trimming or filtering messages in graph state.

We've used these concepts in a Chatbot with memory that produces a running summary of the conversation.

## Goals

But, what if we want our Chatbot to have memory that persists indefinitely?

Now, we'll introduce some more advanced checkpointers that support external databases.

Here, we'll show how to use [Postgres as a checkpointer](https://langchain-ai.github.io/langgraph/how-tos/persistence_postgres/)

In [1]:
%%capture --no-stderr
%pip install -U langgraph langgraph-checkpoint-postgres psycopg psycopg-pool langchain_google_genai


In [2]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [3]:
import os
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"

# Use sync connection¶
This sets up a synchronous connection to the database.

Synchronous connections execute operations in a blocking manner, meaning each operation waits for completion before moving to the next one. The DB_URI is the database connection URI, with the protocol used for connecting to a PostgreSQL database, authentication, and host where database is running. The connection_kwargs dictionary defines additional parameters for the database connection.

In [4]:
from google.colab import userdata
DB_URI = userdata.get('DB_URI')

In [5]:
from psycopg_pool import ConnectionPool
from langgraph.checkpoint.postgres import PostgresSaver

# Connection pool for efficient database access
connection_kwargs = {"autocommit": True, "prepare_threshold": 0}

# Create a persistent connection pool
pool = ConnectionPool(conninfo=DB_URI, max_size=20, kwargs=connection_kwargs)

# Initialize PostgresSaver checkpointer
checkpointer = PostgresSaver(pool)
checkpointer.setup()  # Ensure database tables are set up


Let's re-define our chatbot.

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage

from langgraph.graph import END
from langgraph.graph import MessagesState

model: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", api_key =  GEMINI_API_KEY)

class State(MessagesState):
    summary: str

# Define the logic to call the model
def call_model(state: State) -> State:

    # Get summary if it exists
    summary = state.get("summary", "")
    print(f"Using summary: {summary}")

    # If there is summary, then we add it
    if summary:

        # Add summary to system message
        system_message = f"Summary of conversation earlier: {summary}"

        # Append summary to any newer messages
        messages = [SystemMessage(content=system_message)] + state["messages"]

    else:
        messages = state["messages"]

    response = model.invoke(messages)
    return {"messages": response}

def summarize_conversation(state: State) -> State:
    print(f"Messages before summarizing: {len(state['messages'])}")
    # First, we get any existing summary
    summary = state.get("summary", "")
    print(f"Existing summary: {summary}")

    # Create our summarization prompt
    if summary:

        # A summary already exists
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )

    else:
        summary_message = "Create a summary of the conversation above:"


    # Add prompt to our history
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    # Summarization logic
    print(f"New summary: {response.content}")

    # Delete all but the 2 most recent messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]

    print(f"Messages after truncation: {len(delete_messages)}")
    return {"summary": response.content, "messages": delete_messages}

# Determine whether to end or summarize the conversation
def should_continue(state: State) -> State:

    """Return the next node to execute."""

    messages = state["messages"]
    print(f"Message count: {len(messages)}")
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"

    # Otherwise we can just end
    return END

Now, we just re-compile with our postgres checkpointer.

In [7]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.state import CompiledStateGraph

# Redefine workflow
workflow = StateGraph(State)
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

# Compile the workflow with PostgreSQL checkpointer
graph = workflow.compile(checkpointer=checkpointer)


Now, we can invoke the graph several times.

In [17]:
# Configuration for thread
config = {"configurable": {"thread_id": "4"}}

# Start a conversation
input_message = HumanMessage(content="hi! I'm Areeb")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: 
Message count: 2
================================== Ai Message ==================================

Hi Areeb!  Nice to meet you. How can I help you today?


StateSnapshot(values={'messages': [HumanMessage(content="hi! I'm Areeb", additional_kwargs={}, response_metadata={}, id='dfb4c33c-465e-4fac-9668-9382148e5e94'), AIMessage(content='Hi Areeb!  Nice to meet you. How can I help you today?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-b6efa33a-3dcb-417b-a2d0-112853f40ade-0', usage_metadata={'input_tokens': 9, 'output_tokens': 19, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}})]}, next=(), config={'configurable': {'thread_id': '4', 'checkpoint_ns': '', 'checkpoint_id': '1efbafc2-25b3-610e-8001-06275904d0e0'}}, metadata={'step': 1, 'source': 'loop', 'writes': {'conversation': {'messages': AIMessage(content='Hi Areeb!  Nice to meet you. How can I help you today?\n', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'safety_ratings': [], 'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}},

In [18]:
# Configuration for thread
config = {"configurable": {"thread_id": "4"}}

# Start a conversation
input_message = HumanMessage(content="I like painting pictures.")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: 
Message count: 4
================================== Ai Message ==================================

That's wonderful, Areeb!  What kind of pictures do you like to paint?  Do you have a favorite subject or style?


StateSnapshot(values={'messages': [HumanMessage(content="hi! I'm Areeb", additional_kwargs={}, response_metadata={}, id='dfb4c33c-465e-4fac-9668-9382148e5e94'), AIMessage(content='Hi Areeb!  Nice to meet you. How can I help you today?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-b6efa33a-3dcb-417b-a2d0-112853f40ade-0', usage_metadata={'input_tokens': 9, 'output_tokens': 19, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='I like painting pictures.', additional_kwargs={}, response_metadata={}, id='a9eb58f5-b7ab-4461-9197-a330629cb82f'), AIMessage(content="That's wonderful, Areeb!  What kind of pictures do you like to paint?  Do you have a favorite subject or style?\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-71f7de44-

After that disconnect your runtime and run until you compile your graph.Then,run below code to check whether  state is persisted or not

In [8]:
# Configuration for thread
config = {"configurable": {"thread_id": "4"}}

# Start a conversation
input_message = HumanMessage(content="What's my name and what is my hobby?")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: 
Message count: 6
================================== Ai Message ==================================

Your name is Areeb, and your hobby is painting pictures.


StateSnapshot(values={'messages': [HumanMessage(content="hi! I'm Areeb", additional_kwargs={}, response_metadata={}, id='dfb4c33c-465e-4fac-9668-9382148e5e94'), AIMessage(content='Hi Areeb!  Nice to meet you. How can I help you today?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-b6efa33a-3dcb-417b-a2d0-112853f40ade-0', usage_metadata={'input_tokens': 9, 'output_tokens': 19, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='I like painting pictures.', additional_kwargs={}, response_metadata={}, id='a9eb58f5-b7ab-4461-9197-a330629cb82f'), AIMessage(content="That's wonderful, Areeb!  What kind of pictures do you like to paint?  Do you have a favorite subject or style?\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-71f7de44-

In [12]:
pool.close()

### Persisting state

Using database like Postgres means state is persisted!

For example, we can re-start the notebook kernel and see that we can still load from Postgres DB on disk.
